# PySpark Huggingface Inferencing
### Sentence Transformers

From: https://huggingface.co/sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']


#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [ ]:
embedding

## PySpark

## Inference using Spark ML Model
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [ ]:
import sparkext

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)

In [ ]:
df.show()

In [ ]:
my_model = sparkext.huggingface.SentenceTransformerModel(model) \
                .setInputCol("lines") \
                .setOutputCol("embedding")

In [ ]:
embeddings = my_model.transform(df)

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
results[0]

## Inference using Spark DL UDF
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

### Using model instance on driver

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(200)

In [ ]:
df.schema

In [ ]:
encode = sentence_transformer_udf(model)

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
results[0]

### Using model_id string on driver

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(200)

In [ ]:
encode = sentence_transformer_udf("paraphrase-MiniLM-L6-v2")

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
results[0]

### Using model loader

In [ ]:
from pyspark.sql.functions import col
from sparkext.huggingface import sentence_transformer_udf

In [ ]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(200)

In [ ]:
def model_loader(model_name):
    from sentence_transformers import SentenceTransformer
    return SentenceTransformer(model_name)   

In [ ]:
encode = sentence_transformer_udf("paraphrase-MiniLM-L6-v2", model_loader=model_loader)

In [ ]:
embeddings = df.withColumn("encoding", encode(col("lines")))

In [ ]:
%%time
results = embeddings.collect()

In [ ]:
results[0]